# コード実行例
prgフォルダのパスは適宜変更して実行してください。  
SASのエンコーディングはUTF-8で実行してください。  

動作は基本的にSAS Ondemandを利用して確認しました。SASのバージョンは9.4であれば動作するかと思います。  

## 注意点
windowsのローカルでも動作は問題ないかと思いますが、バイナリ関連のフォーマット等を利用しているため、実行環境によってはうまく動作しない可能性があります。  
欠損への対応や、sashelp.carsなどある程度のサイズのデータセットでも問題ないことは確認していますが、それほど多くのテストは行われていません。  
レコード数が多い場合などで、特にparquetへの変換ではエンコードの対応が十分でない可能性があることには注意してください。

## 変換仕様の補足
仕様からだと分かりづらい部分は、pyarrowで作成されるファイルを参考にしてプログラムを作成しました。  
SASで変換を行うにあたり、仕様の範囲で効率よりも扱いが簡単なように変換を行っている箇所があります。  
例として、Parquetではdefinition levelに対して、RLEを使わずにBit-Packingのみで対応していたり、Arrow IPCでは型が同じ変数で共用可能なvtableを個別にもたせていたり、というような点があります。  
そのため変換されたファイルは、pyarrowやpandasなどから作成したファイルとは、データの中身は同じでもバイナリで比較すると異なる箇所があります。

## SASのみで実行する場合
文章用にjupyterを使用していますが、以下のようにSASのみで実行可能です。

```sas
%include prg/*.sas ;
sd2pq(table=class, libref=sashelp, outpath=C:\sh_class.parquet) ;
```

# pythonからの実行と確認

以下では変換とその結果まで確認できるように、saspyから実行します。  
ここではsashelp.classをこのipynbファイルと同じフォルダに出力し、読み込みを行います。  

汎用的な動作とテスト用環境がサーバーのSASを利用していることからsas.downloadを使用していますが、ローカルでアクセス可能ならそのまま扱っても問題ありません。

In [ ]:
ipc_file = 'sh_class.arrow'
prg_path = 'prg'

import saspy
sas = saspy.SASsession()
sas_code = f"""
%let ipc_path=%sysfunc(pathname(work))/{ipc_file} ;
%include "{prg_path}/*.sas" ;
%sd2IPC(table=class, libref=sashelp, outpath=&ipc_path.) ;
"""

ll = sas.submit(sas_code)
remote_file = sas.symget('ipc_path')
local_file = ipc_file

sas.download(local_file, remote_file)
sas.endsas()

import pyarrow as pa
class_df = pa.ipc.open_file(local_file).read_all()
print(class_df)

Using SAS Config named: oda
SAS Connection established. Subprocess id is 36292

SAS Connection terminated. Subprocess id was 36292
pyarrow.Table
Name: string
Sex: string
Age: double
Height: double
Weight: double
----
Name: [["Alfred","Alice","Barbara","Carol","Henry",...,"Philip","Robert","Ronald","Thomas","William"]]
Sex: [["M","F","F","F","M",...,"M","M","M","M","M"]]
Age: [[14,13,13,14,14,...,16,12,15,11,15]]
Height: [[69,56.5,65.3,62.8,63.5,...,72,64.8,67,57.5,66.5]]
Weight: [[112.5,84,98,102.5,102.5,...,150,128,133,85,112]]
